# Classification task using late merge with hard paramter sharing and adaptive learning rates #

in this model wwe will be using adaptive learning rates and reduce the learning rate as the model trains on a time based decay strategy using a learning rate scheduler the network architecture will be a late merge with hard parameter sharing as in my experience it is the best for this tak, i also tried soft paramter sharing wiht eraly merge but that was not as accurate in this case

In [21]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import  Dataset
from PIL import Image
from torchvision import transforms
import random
from sklearn.metrics import accuracy_score
import torch.optim.lr_scheduler as lr_scheduler

In [22]:
class lmerge_dataloader(Dataset):
    def __init__(self, path , train):
        self.path = path
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.data_lister(train)
        self.length = len(self.dataset)
    def data_lister(self, train): 
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.path, class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        random.shuffle(data_list)
        if train:
            return data_list[:int(0.8*len(data_list))]
        else:
            return data_list[int(0.8*len(data_list)):]
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
      
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png')).convert('L')) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '1.png')).convert('L')) # image 2 in tensor
      
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
        ])
        pic1 = transform(pic1)
        pic2 = transform(pic2)
        
        return pic1 , pic2, sample_class

In [23]:
adaptive_training = lmerge_dataloader('./dataset/', True)
adaptive_testing = lmerge_dataloader('./dataset/', False)

In [24]:
sample8 = adaptive_training[1311]
print(sample8[0].shape)
print(sample8[1].shape)
print(sample8[2])

torch.Size([1, 224, 224])
torch.Size([1, 224, 224])
0


In [25]:
class late_merge_NN(nn.Module):
    def __init__(self):
        super(late_merge_NN, self).__init__()
        self.convoluton1 = nn.Conv2d(1, 2, 1)
        self.convoluton2 = nn.Conv2d(2, 4, 1)
        self.pool = nn.MaxPool2d(kernel_size=4, stride=5)
        self.convoluton12 = nn.Conv2d(1, 2, 1)
        self.convoluton22 = nn.Conv2d(2, 4, 1)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=5)
        self.full1 = nn.Linear(648, 2)

    def forward(self, x1, x2):
        x1 = self.convoluton1(x1)
        x1 = self.pool(x1)
        x1 = self.convoluton2(x1)
        x1 = self.pool(x1)
        x1 = x1.view(-1, 324)
        
        x2 = self.convoluton12(x2)
        x2 = self.pool2(x2)
        x2 = self.convoluton22(x2)
        x2 = self.pool2(x2)
        x2 = x2.view(-1, 324)

        x = torch.cat((x1, x2), dim=1)
        x = self.full1(x)
        return x

In [26]:
# hyper parameters
learning_rate_8= 0.01
epochs_8 = 3

In [27]:
# training the neural net with the stacking dataset
model8 = late_merge_NN()
criterion8 = nn.CrossEntropyLoss()
optimiser8 = optim.Adam(model8.parameters(), lr = learning_rate_8)
scheduler = lr_scheduler.StepLR(optimiser8, step_size=1, gamma=0.5)

In [28]:
# model params
total_params = sum(p.numel() for p in model8.parameters())
print(f'the total number of parameters in this model are {total_params}')

the total number of parameters in this model are 1330


In [29]:
model8.train()
for epoch in range(epochs_8):
    running_loss = 0 
    predicted_list = []
    labels_list = []
    print(f'this is epoch number {epoch}')
    for i, (input1,input2, label) in enumerate(adaptive_training):
        optimiser8.zero_grad()
        labels_list.append(label)
        label = torch.tensor(label).view(-1)
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model8(input1, input2)
        loss = criterion8(outputs, label)
        loss.backward()
        running_loss += loss.item()
        optimiser8.step()
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.item())
        if i % 1000 == 999:   
            print('loss: %.3f accuracy: %.3f' %(running_loss / 10, 100 * accuracy_score(labels_list, predicted_list)))
            running_loss = 0
    scheduler.step()

this is epoch number 0
this is the 0 iteration
loss: 48.118 accuracy: 82.100
this is the 1000 iteration
loss: 28.420 accuracy: 85.750
this is the 2000 iteration
this is epoch number 1
this is the 0 iteration
loss: 23.916 accuracy: 90.800
this is the 1000 iteration
loss: 22.959 accuracy: 91.300
this is the 2000 iteration
this is epoch number 2
this is the 0 iteration
loss: 18.408 accuracy: 93.500
this is the 1000 iteration
loss: 17.379 accuracy: 93.850
this is the 2000 iteration


In [30]:
model8.eval()
loss = 0 
predicted_list = []
labels_list = []
with torch.no_grad():
    for i, (input1,input2, label) in enumerate(adaptive_testing):
        label = torch.tensor(label)
        label = torch.tensor(label).view(-1)
        labels_list.append(label.item())
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model8(input1, input2)
        loss += criterion8(outputs, label).item()
        predicted = outputs.argmax(dim = 1 , keepdim = True)
        predicted_list.append(predicted.item())


/var/folders/l8/nqt3x5yn28z6g3mdkkcd_w940000gn/T/ipykernel_29060/4184472181.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).view(-1)


this is the 0 iteration


In [31]:
print(f'the model accuracy is {accuracy_score(labels_list, predicted_list)}')

the model accuracy is 0.9425770308123249
